# Iceberg Detection and Tracking Pipeline

This notebook demonstrates the complete workflow for detecting and tracking icebergs across time-lapse imagery.

## Pipeline Overview

The pipeline consists of five main stages:

1. **Embedding Training**: Train a Vision Transformer-based Siamese network for appearance similarity
2. **Detection Training**: Train a Faster R-CNN model to detect icebergs in images
3. **Detection Inference**: Apply the trained detector to new sequences
4. **Tracking**: Associate detections across frames using appearance and motion features
5. **Visualization**: Generate annotated imagery and videos of tracked icebergs
---

## Dataset Configuration

Specify the training and test datasets. These should correspond to directories in your data structure.

#### Basic Directory Structure
```
data/
└── hill/
    ├── train/
    │   ├── images/           # Training images
    │   └── ground_truth/
    │       └── gt.txt        # MOT format annotations (required for training)
    └── test/
        └── images/           # Test images (inference only)
```

#### Optional Features

**Multiple Sequences**: Organize different conditions into subfolders:
```
hill/
├── train/
│   ├── melange/
│   │   ├── images/
│   │   └── ground_truth/
│   └── night/
│       ├── images/
│       └── ground_truth/
```

**Region Masking**: Exclude irrelevant areas via a masking image with black pixels at irrelevant areas (e.g. sky, land, glacier). Black pixels in the mask will be ignored during processing. Place masking image at ``hill/mask.<image_format>``

#### Ground Truth Format (Training Only)

Annotations follow the [MOT Challenge](https://motchallenge.net/instructions/) format—a CSV file where each line represents one object:
```
<frame>, <id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <conf>, <x>, <y>, <z>
```

**Example:**
```csv
_MG_17310,1,161.15,1087.59,247.32,90.46,1.0,1,-1,-1
_MG_17310,2,2113.04,2085.49,248.25,97.31,1.0,1,-1,-1
```

- `<frame>`: Image filename
- `<id>`: Unique object ID
- `<bb_left>, <bb_top>, <bb_width>, <bb_height>`: Bounding box coordinates
- `<conf>`: Confidence score (typically 1.0 for ground truth)
- `<x>, <y>, <z>`: Additional coordinates (use -1 if unused)

**Note**: Ground truth is only required for training custom models. Inference on test data does not require annotations.


---



## Setup and Imports

Import all necessary modules for the pipeline.

In [ ]:
from detection import IcebergDetector, IcebergDetectionConfig
from embedding import IcebergEmbeddingsTrainer, IcebergEmbeddingsConfig
from tracking import IcebergTracker, IcebergTrackingConfig
from utils.visualize import Visualizer, VisualizationConfig
from utils.helpers import load_config

In [1]:
# Define dataset paths
train_dataset = "hill/train"
test_dataset = "hill/test"

---

## Stage 1: Embedding Training

Train a Vision Transformer-based Siamese network to learn appearance similarity between iceberg patches. This model will be used during tracking to re-identify icebergs across frames based on their visual appearance.

#### Key Parameters:
- **Architecture**: Vision Transformer (ViT) backbone
- **Training objective**: Contrastive learning with triplet loss
- **Output**: Feature embeddings for appearance-based matching

#### Output:
- Embedding model saved to `models/iceberg_detection_model.pth`
- Ground truth embeddings saved to `data/{dataset}/ground_truth/`

In [ ]:
# Initialize embedding trainer with default configuration
config = IcebergEmbeddingsConfig(dataset=train_dataset)
trainer = IcebergEmbeddingsTrainer(config)

# Run the complete training pipeline
trainer.run_complete_pipeline()

#### Configuration Options:
Each pipeline stage can be configured in two ways. You can either:
1. **Use Default parameters**: Instantiate config dataclasses directly (as shown in the code cells)
2. **Load from YAML configuration files**: Load settings from `configs/*.yaml` files using `load_config()` as follows:
    ```python
    # YAML approach (alternative)
    config = load_config(cfg_file="configs/detect.yaml", dataset=train_dataset)

    # You can also override specific parameters:
    config = IcebergTrackingConfig(
        dataset=test_dataset,
        kalman_distance_weight=0.9,
        min_iceberg_id_count=10
    )

    ```

All subsequent code cells use the default approach for simplicity, but you can substitute with YAML configs as needed.

---

## Stage 2: Detection Training

Train a Faster R-CNN model to detect icebergs in images. This model learns to localize icebergs and predict bounding boxes.

#### Key Parameters:
- **Architecture**: Faster R-CNN with ResNet-50-FPN backbone
- **Training objective**: Object detection with bounding box regression
- **Output**: Trained detector checkpoint

#### Output:
- Detection model saved to `models/iceberg_detection_model.pth`

In [ ]:
# Initialize detector with training dataset
config = IcebergDetectionConfig(dataset=train_dataset)
detector = IcebergDetector(config)

# Train the detection model
detector.train()

---

## Stage 3: Detection Inference

Apply the trained Faster R-CNN model to detect icebergs in the test dataset. This generates bounding box predictions for all images in the sequence.

#### Output:
- Detection results and embeddings saved to `data/{dataset}/detections/`
- Format: MOT-format text and PyTorch file

In [ ]:
# Initialize detector with test dataset
config = IcebergDetectionConfig(dataset=test_dataset)
detector = IcebergDetector(config)

# Run inference on test images
detector.predict()

---

## Stage 4: Tracking

Associate detections across frames to create continuous trajectories for each iceberg. The tracker uses:

#### Tracking Features:
- **Appearance similarity**: Vision Transformer embeddings from Stage 1
- **Motion prediction + distance**: Kalman filtering for position and velocity estimation
- **Size consistency**: Iceberg size similarity across frames

#### Matching Strategy:
- Similarity metrics weighted combination of appearance, distance (incl Kalman) and size
- Bidirectional matching to reduce ID switches
- Adaptive thresholds based on time intervals
- Track lifecycle management to track not matched icebergs

#### Output:
- Tracking results saved to `data/{dataset}/tracking/`
- Format: MOT-format text files with frame-by-frame track assignments

In [ ]:
# Initialize tracker with test dataset
config = IcebergTrackingConfig(dataset=test_dataset)
tracker = IcebergTracker(config)

# Run tracking algorithm
tracker.track()

---

## Stage 5: Visualization

Generate visual outputs showing tracked icebergs with unique IDs and trajectories.

#### Visualization Options:
1. **Annotated images**: Individual frames with bounding boxes and track IDs
2. **Video rendering**: Complete sequence compiled into a video (optional)

#### Output:
- Annotated images saved to `data/{dataset}/visualizations/`
- Optional video saved to `data/{dataset}/videos/`

In [ ]:
# Initialize visualizer with test dataset
config = VisualizationConfig(dataset=test_dataset)
visualizer = Visualizer(config)

# Generate annotated images
visualizer.annotate_icebergs()

# Optionally render video (uncomment to enable)
# visualizer.render_video()

---

## Pipeline Complete! 🎉

#### Output Directory Structure

After running the complete pipeline, your directory will contain:

```
data/{dataset}/
├── images/                         # Original images
├── ground_truth/                   # Ground truth annotations (only necessary for training or performance evaluation)
├── detections/                     # Detection results (Stage 3)
├── tracking/                       # Tracking results (Stage 4)
└── visualizations/                 # Annotated images (Stage 5)
models/
├── iceberg_embedding_model.pth     # Training results (Stage 1)
├── iceberg_detection_model.pth     # Training results (Stage 2)
└── sam_vit_b_01ec64.pth            # Automatically downloaded, if icebergs contours should be visualized (Stage 5)
```

#### What's Next?

1. **Evaluate Results**: Use TrackEval metrics to assess tracking performance

2. **Adjust Parameters**: Fine-tune configuration files in `configs/` directory:
   - `embed.yaml`: Embedding model hyperparameters
   - `detect.yaml`: Detection model settings
   - `track.yaml`: Tracking algorithm parameters
   - `visualize.yaml`: Visualization options

   Alternatively, you can fine-tune parameters directly by passing arguments to the config dataclasses:
```python
   config = IcebergTrackingConfig(
       dataset=test_dataset,
       weight_appearance=0.7,
       min_iceberg_id_count=10,
   )
   config.get_gt_thresholds = False
```